In [2]:
import numpy as np
import matplotlib.pyplot as plt 
from itertools import combinations, product
from matplotlib.colors import Normalize

Rotation of the stiffness tensor (orthorhombic lattice). The initial stiffness tensor corresponds to ZrB₂ in $P6/mmm$ lattice within $[11\bar{2}0]-[\bar{1}100]-[0001]$ coordinate system.

In [6]:
cij = np.load('data/ZrB2_cij_3.172_3.546_try_1_fin.npy')
cij

array([[ 5.64198172e+02,  1.28288064e+02,  5.34288205e+01,
         6.01472355e-05, -3.02534032e-05, -9.98973357e-07],
       [ 1.28288064e+02,  4.00956533e+02,  1.28311756e+02,
         1.19407271e-04, -6.93050609e-05,  1.48833230e-05],
       [ 5.34288205e+01,  1.28311756e+02,  5.64177139e+02,
        -6.56355432e-05,  5.79016134e-06,  3.71579354e-06],
       [ 4.89702765e-04,  4.74991165e-04, -6.18306861e-04,
         2.52387177e+02,  1.06245621e-05,  3.13085710e-05],
       [ 2.20091130e-05, -6.98742709e-05,  2.25640038e-04,
         1.89443491e-06,  2.55468110e+02, -3.79005762e-05],
       [ 8.55184603e-05, -6.00820808e-05, -1.60032364e-04,
         5.90478480e-05,  2.05481666e-05,  2.52386518e+02]])

In [11]:
def uvtw2uvw(vec_uvtw):
    uvtw_basemat = np.array([
        [1, 0, 0],
        [0, 1, 0],
        [-1, -1, 0],
        [0, 0, 1]
    ])
    return uvtw_basemat.T@vec_uvtw

def uvw2uvtw(vec_uvw):
    uvtw_basemat = np.array([
        [1, 0, 0],
        [0, 1, 0],
        [-1, -1, 0],
        [0, 0, 1]
    ])
    return np.linalg.pinv(uvtw_basemat.T)@vec_uvw

def a_rot(psi, phi, theta=0):
    return np.array([
        
        [np.cos(theta)*np.cos(psi)*np.cos(phi)-np.sin(theta)*np.sin(phi), 
         np.cos(theta)*np.cos(psi)*np.sin(phi)+np.sin(theta)*np.cos(phi), 
         -np.cos(theta)*np.sin(psi)],
        
        [-np.sin(theta)*np.cos(psi)*np.cos(phi)-np.cos(theta)*np.sin(phi), 
         -np.sin(theta)*np.cos(psi)*np.sin(phi)+np.cos(theta)*np.cos(phi), 
         np.sin(theta)*np.sin(psi)],
        
        [np.sin(psi)*np.cos(phi), 
         np.sin(psi)*np.sin(phi), 
         np.cos(psi)],
        
    ])

def ijkl_invert(sij_voigt):
    
    sijkl = np.zeros((3,3,3,3))
    for i, j, k, l in product(range(3), repeat=4):
        ij_ind, kl_idn = comb_dict[(i,j)], comb_dict[(k,l)]
        
        if ij_ind < 3 and kl_idn < 3:
            sijkl[i,j,k,l] = sij_voigt[ij_ind, kl_idn]
        elif np.min([ij_ind, kl_idn]) < 3 and np.max([ij_ind, kl_idn]) >= 3:
            sijkl[i,j,k,l] = sij_voigt[ij_ind, kl_idn]/2
        elif ij_ind >= 3 and kl_idn >= 3: 
            sijkl[i,j,k,l] = sij_voigt[ij_ind, kl_idn]/4
            
    return sijkl
        
cij_0001 = np.load('data/ZrB2_cij_3.172_3.546_try_1_fin.npy')
# cij_0001 = np.clip(cij_0001, 0, None)
sij_0001 = np.linalg.inv(cij_0001)

a, c = 3.172, 3.546
#* 1000-0001-0110 lattice vector
lattice_vec = np.array([
    [a, 0, 0],
    [0, c, 0],
    [0, 0, np.sqrt(3)*a],
])

#TODO angle method-tensor
# psi, phi, theta = 0, np.arctan(c/a), 0
# rot_tensor = a_rot(psi, phi, theta)
#* ====== angle-based method ======

#TODO vector method-tensor
#* raw vector in Cartesian coordinate
#* 1000-0001-0110 1100 coordinate system
nmt_raw = (np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1],
])@lattice_vec)
nmt_raw = nmt_raw/np.linalg.norm(nmt_raw, axis=1)[:,None]

#* new vector in Cartesian coordinate
#* 11-20 - -1100 - 0001 coordinate system
nmt_n = np.array([
    [a/np.sqrt(a**2+c**2), c/np.sqrt(a**2+c**2), 0],
    [-c/np.sqrt(a**2+c**2), a/np.sqrt(a**2+c**2), 0],
    [0,0,1],
])
#* 1100 - 000-1 - -1100 coordinate system
# nmt_n = np.array([
#     [a/np.sqrt(a**2+c**2), c/np.sqrt(a**2+c**2), 0],
#     [0, 0, -1],
#     [-c/np.sqrt(a**2+c**2), a/np.sqrt(a**2+c**2), 0],
# ])
rot_tensor = nmt_n@np.linalg.inv(nmt_raw)
#* ======== vector-based method ========

print('Orthogonality check:\n', np.round(rot_tensor@rot_tensor.T, 2))

comb_dict = {
    (0,0):0,
    (0,1):5,
    (1,0):5,
    (0,2):4,
    (2,0):4,
    (1,1):1,
    (1,2):3,
    (2,1):3,
    (2,2):2,
}

sij_1100_ijkl = np.zeros((3,3,3,3))
for i, j, k, l in product(range(3), repeat=4):
    for m, n, o, p in product(range(3), repeat=4):
        
        aim = rot_tensor[i, m]
        ajn = rot_tensor[j, n]
        ako = rot_tensor[k, o]
        alp = rot_tensor[l, p]
        
        mn_ind = comb_dict[(m,n)]
        op_ind = comb_dict[(o,p)]
        ij_ind = comb_dict[(i,j)]
        kl_ind = comb_dict[(k,l)]
        
        sij_0001_ijkl = ijkl_invert(sij_0001)
        sij_1100_ijkl[i, j, k, l] += aim*ajn*ako*alp*sij_0001_ijkl[m, n, o, p]

print(f'raw shear modulus: {1/sij_0001[5,5]}')
print(f'raw poisson ratio: {-sij_0001_ijkl[0,0,1,1]/sij_0001_ijkl[0,0,0,0]}')

Orthogonality check:
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
raw shear modulus: 252.38651799999883
raw poisson ratio: 0.3123847834611543


In [12]:
def ijkl2voigt(sijkl):
    sij_voigt = np.zeros((6,6))
    for i,j,k,l in product(range(3), repeat=4):
        ij_ind, kl_ind = comb_dict[(i,j)], comb_dict[(k,l)]
        sij_voigt[ij_ind, kl_ind] += sijkl[i,j,k,l]
    return sij_voigt

sij_1100 = ijkl2voigt(sij_1100_ijkl)
cij_1100 = np.linalg.inv(sij_1100)
print(f'new shear modulus: {1/sij_1100[5,5]}')
print(f'new poisson ratio: {-sij_1100[0,1]/sij_1100[0,0]}')
sij_1100, cij_1100

new shear modulus: 167.81334470832172
new poisson ratio: 0.05041537863247894


(array([[ 1.94878863e-03, -9.82489168e-05, -3.52083944e-04,
         -1.07377569e-09, -7.91770203e-10,  5.90473014e-04],
        [-9.82488219e-05,  1.84179971e-03, -2.90772753e-04,
         -5.53224904e-10,  1.55711788e-10,  3.67448675e-04],
        [-3.52084481e-04, -2.90772217e-04,  1.91269377e-03,
          6.84459914e-10,  4.48932277e-10, -5.48956215e-04],
        [-3.76778862e-09, -2.64630048e-09,  5.31047110e-09,
          3.93562281e-03,  2.37440726e-05, -3.14197018e-09],
        [-2.82789164e-09, -2.72581249e-09,  3.16339379e-09,
          2.37442080e-05,  3.94092652e-03, -1.14436337e-09],
        [ 5.90472258e-04,  3.67449453e-04, -5.48956335e-04,
         -1.23275833e-09, -3.44665606e-10,  5.95900166e-03]]),
 array([[ 5.47832406e+02,  5.39732356e+01,  9.50260813e+01,
          1.24668917e-04,  9.20831332e-05, -4.88584338e+01],
        [ 5.39732241e+01,  5.65951967e+02,  8.67144952e+01,
          6.92444023e-05, -2.46343874e-05, -3.22580161e+01],
        [ 9.50262441e+01,  8.6